<a href="https://colab.research.google.com/github/hchristiaan/TrainCNN4SignatureVerification/blob/master/ProjectCode_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Please note that:
#     # - Means comment explaining the code
#     ### - Means things that need to be addressed

In [0]:
# Import all required repositories for the project
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

import keras
import keras.backend as K
import tensorflow as tf
from keras import applications
from keras.models import Model
from keras.layers import Flatten, Dense, Input,concatenate
from keras.optimizers import Adam
from keras.models import load_model, model_from_json

import random

Using TensorFlow backend.


In [0]:
# Import the google drive to upload the data
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Unzip the image dataset
!unzip '/content/drive/Team Drives/inputdataset/UTSig.zip' -d /content

Archive:  /content/drive/Team Drives/inputdataset/UTSig.zip
   creating: /content/UTSig/Forgery/
   creating: /content/UTSig/Forgery/Opposite Hand/
   creating: /content/UTSig/Forgery/Opposite Hand/1/
  inflating: /content/UTSig/Forgery/Opposite Hand/1/1.tif  
  inflating: /content/UTSig/Forgery/Opposite Hand/1/2.tif  
  inflating: /content/UTSig/Forgery/Opposite Hand/1/3.tif  
   creating: /content/UTSig/Forgery/Opposite Hand/10/
  inflating: /content/UTSig/Forgery/Opposite Hand/10/1.tif  
  inflating: /content/UTSig/Forgery/Opposite Hand/10/2.tif  
  inflating: /content/UTSig/Forgery/Opposite Hand/10/3.tif  
   creating: /content/UTSig/Forgery/Opposite Hand/100/
  inflating: /content/UTSig/Forgery/Opposite Hand/100/1.tif  
  inflating: /content/UTSig/Forgery/Opposite Hand/100/2.tif  
  inflating: /content/UTSig/Forgery/Opposite Hand/100/3.tif  
   creating: /content/UTSig/Forgery/Opposite Hand/101/
  inflating: /content/UTSig/Forgery/Opposite Hand/101/1.tif  
  inflating: /content/UT

In [0]:
# Proposed file restructing to fit this code
import os, shutil

filePath = '/content/UTSig'

filePathGen = filePath + '/Genuine'
filePathFor = filePath + '/Forgery'
filePathForSk = filePath + '/Forgery/Skilled' 
filePathForOh = filePath + '/Forgery/Opposite Hand' 
filePathForSi = filePath + '/Forgery/Simple' 

if "Simple" in os.listdir(filePathFor):
  if "1" in os.listdir(filePathGen):
      for i in os.listdir(filePathGen):
          author = i
          for j in os.listdir(filePathGen + '/' + author):
              os.rename(filePathGen + '/' + author + "/" + j, filePathGen + "/" + "g_" + author + "_" + j)
          os.removedirs(filePathGen + "/" + author)

  if "Skilled" in os.listdir(filePathFor):
      for i in os.listdir(filePathForSk):
          author = i
          for j in os.listdir(filePathForSk + '/' + author):
              os.rename(filePathForSk + '/' + author + "/" + j, filePathFor + "/" + "fSk_" + author + "_" + j)
          os.removedirs(filePathForSk + "/" + author)

  if "Opposite Hand" in os.listdir(filePathFor):
      for i in os.listdir(filePathForOh):
          author = i
          for j in os.listdir(filePathForOh + '/' + author):
              os.rename(filePathForOh + '/' + author + "/" + j, filePathFor + "/" + "fOh_" + author + "_" + j)
          os.removedirs(filePathForOh + "/" + author)

  if "Simple" in os.listdir(filePathFor):
      shutil.rmtree(filePathForSi)

In [0]:
### Need code for altering .tif files into .png? or can we go ahead and just work with .tif? ###

In [0]:
# File paths of the image data used later in the project
### Need to discuss how we will seperate our data. 80% 20% rule? ###
gentr="/content/UTSig/Genuine"
forgtr="/content/UTSig/Forgery"

gent="/content/UTSig/Genuine"
forgt="/content/UTSig/Forgery"

In [0]:
# Code block for working with the image files (resizing, retreiving files, etc.)

# Standard height weight for images
img_width, img_height, channels = 224, 224, 3

dim = (img_width, img_height)

# Converts the images to rgb which is what VGG-16 takes as input
def to_rgb(img):
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 
    img_rgb = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    return img_rgb


# Takes in file path and image name and returns resized image
def returnimages(path,img):
    image=cv2.imread(path+"/"+ img)                  #bringing the image
    image=cv2.resize(image, (img_width, img_height))
    image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image=to_rgb(image).reshape(1,img_width, img_height,3)/255.0       #resizing and normalizing    
    return image     

# Takes in number of author, path for genuine and forged signatures
# Returns one image name (m.pop()), and two arrays of file names n = forgary, m = genuine
# Used for generator()
def getfiles(num,gen,forg):
    genList=os.listdir(gen)
    forgList=os.listdir(forg)
    author=str(num)
           
    forgArr,genArr=[],[]
    for i in forgList:
        if ("_" + author + "_") in i:
            forgArr = forgArr + [i]
            
    for i in genList:
        if ("_" + author + "_") in i:
            genArr = genArr + [i]
     
    return genArr.pop(), forgArr, genArr
######################################

# Same as getfiles but returns two files
def getfilest(num,gen,forg):
    genList=os.listdir(gen)
    forgList=os.listdir(forg)
    author=str(num)
           
    forgArr,genArr=[],[]
    for i in forgList:
        if ("_" + author + "_") in i:
            forgArr = forgArr + [i]
            
    for i in genList:
        if ("_" + author + "_") in i:
            genArr = genArr + [i]

    return genArr.pop(), genArr.pop(), forgArr, genArr

In [0]:
# Function for defining loss variable in model, leave for now, look into altering?
### Unsure what y_pred is doing, or how this works....
def triplet_loss(y_true, y_pred):
    alpha = 0.5
    anchor, positive, negative =y_pred[0,0:512], y_pred[0,512:1024], y_pred[0,1024:1536]
    
    positive_distance = K.mean(K.square(anchor - positive),axis=-1)
    negative_distance = K.mean(K.square(anchor - negative),axis=-1)
    return K.mean(K.maximum(0.0, positive_distance - negative_distance + alpha))

In [0]:
# Defining the architecture. chose to go with 16 as it is smaller...
model = applications.vgg16.VGG16(weights='imagenet', include_top=False, pooling='max')

### Imput baseline model ### Unsure how this is done, commenting for now
baseModel = applications.vgg16.VGG16(weights=None, include_top=False, pooling='max')

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 1s 0us/step


In [0]:
# For layers from 1 to 15 freeze the weights

### We will need to do a test for each layer to find the optimal transfer* ###
### For now just copying over the code from example
for layer in model.layers[:5]:
    layer.trainable = False

In [0]:
# all "in" variables are defining the input "shape" for the model
# Input is a function in Keras
# Positive represents genuine / Negative represents Forgary 
anchor_in = Input(shape=(img_width, img_height, channels))
pos_in = Input(shape=(img_width, img_height, channels))
neg_in = Input(shape=(img_width, img_height, channels))

# Defining output of model
anchor_out = model(anchor_in)
pos_out = model(pos_in)
neg_out = model(neg_in)
merged_vector = concatenate([anchor_out, pos_out, neg_out],axis=1)

In [0]:
# Setting inputs and outputs of the model
# Model function - potentially overwrites the input and output layers
### Unsure what the output is... Need to research ###
# Input is 3 images: image in question (anchor), postitive image, negative image (same authors)

model = Model(inputs=[anchor_in, pos_in, neg_in], outputs=merged_vector)
baseModel = Model(inputs=[anchor_in, pos_in, neg_in], outputs=merged_vector)

In [0]:
# Compiling the model
# loss function set to function defined earlier
# Optimizer is for learning rate of the function
### unsure what Adam function does

model.compile(optimizer=Adam(lr=0.000005),loss=triplet_loss)
baseModel.compile(optimizer=Adam(lr=0.000005),loss=triplet_loss)

In [0]:
# Generator - Goes through every author (in test) and returns an anchor (last image) and an array of file names (negative=forged) (positive=genuine) 
def generator():
    for i in range(1,96):
      # Returns last genuine image, array of pos/genuine files, array of neg/forgary files
        anc,neg,pos=getfiles(i,gentr,forgtr)
        
#When we have the list of images/arrays for that author, we then loop through for the length of the negative list (total number of forged)
  #For each negative image, we are going to loop through the length of the positive list
    #For each positive image we resize the anchor, first
    
        for i in range(len(neg)):           
            for j in range(len(pos)):        
                anchor=returnimages(gentr,anc)
                positive=returnimages(gentr,pos[j])
                negative=returnimages(forgtr,neg[i])
               # yield ([anc,pos[j],neg[i]],[0])
                yield ([anchor,positive,negative],[0])

In [0]:
### These values were taken from example, need to decide on what they should be

for x in range(2):
    model.fit_generator(generator(),steps_per_epoch=200,epochs=3)
    
for x in range(2):
    baseModel.fit_generator(generator(),steps_per_epoch=200,epochs=3)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/3
200/200 [==============================] - 22s 112ms/step - loss: 0.0168
Epoch 2/3
200/200 [==============================] - 16s 82ms/step - loss: 0.0094
Epoch 3/3
200/200 [==============================] - 16s 82ms/step - loss: 0.0072
Epoch 1/3
200/200 [==============================] - 17s 83ms/step - loss: 0.0000e+00
Epoch 2/3
200/200 [==============================] - 17s 83ms/step - loss: 0.0000e+00
Epoch 3/3
200/200 [==============================] - 17s 84ms/step - loss: 5.3321e-04


In [0]:
### Unsure purpose of this part ###

# used for validation

tneg,tpos = 0, 0
# Uses values not used in generator 2 to validate
x= range(93,116)

for k in x: #the id of signatures you want to check
    #print("When k is ", k)
    anc,neg,pos=getfiles(k,gent,forgt)
    tneg=tneg+len(neg)
    tpos=tpos+len(pos)
print("Forgary Image Total: " + str(tneg),"Genuine Image Total: " + str(tpos))

Forgary Image Total: 207 Genuine Image Total: 598


In [0]:
os.listdir('/content')

['.config', 'drive', 'UTSig', 'sample_data']

In [0]:
model.save_weights('model_weights.h5')
baseModel.save_weights('baseModel_weights.h5')
os.listdir('/content')

['.config', 'drive', 'UTSig', 'model_weights.h5', 'sample_data']

In [0]:
forg_passed=0
gen_flagged=0
x= range(93,116)

for k in x: #the id of signatures you want to check
    print("When k is ", k)
    anc,neg,pos=getfiles(k,gent,forgt)
    
    anchor=returnimages(gent,anc)
    for i in range(len(pos)): #pos
        positive=returnimages(gent,pos[i])
        x=model.predict([anchor,positive,anchor])
        a, p, useless = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-p)
        dist=np.linalg.norm(a-p)
        #print("positive distance is ",dist)
        if(dist>32):
        #  print("0")
            gen_flagged=gen_flagged+1
            print("gen flagged - ",dist, "file name is - ", pos[i])
            
        else:
            gen_flagged=gen_flagged
        #   print("1")
        
    for j in range(len(neg)): #neg
        negative=returnimages(forgt,neg[j])
        x=model.predict([anchor,negative,anchor])
        a, n, useless = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-n)
        dist=np.linalg.norm(a-n)
        #print("negative distance is ",dist)
        if(dist>32):
            forg_passed=forg_passed
          #  print("0")
        else:
            forg_passed=forg_passed+1
            print("forg passed - ",dist, "file name is - ", neg[j])
          #  print("1")

print("forg_passed is ",forg_passed)
print("gen_flagged is ",gen_flagged)

### Code for base model - duplicated with variables changed ###
forg_passed=0
gen_flagged=0
x= range(93,116)

for k in x: #the id of signatures you want to check
    print("When k is ", k)
    anc,neg,pos=getfiles(k,gent,forgt)
    
    anchor=returnimages(gent,anc)
    for i in range(len(pos)): #pos
        positive=returnimages(gent,pos[i])
        x=baseModel.predict([anchor,positive,anchor])
        a, p, useless = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-p)
        dist=np.linalg.norm(a-p)
        #print("positive distance is ",dist)
        if(dist>32):
        #  print("0")
            gen_flagged=gen_flagged+1
            print("gen flagged - ",dist, "file name is - ", pos[i])
            
        else:
            gen_flagged=gen_flagged
        #   print("1")
        
    for j in range(len(neg)): #neg
        negative=returnimages(forgt,neg[j])
        x=baseModel.predict([anchor,negative,anchor])
        a, n, useless = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-n)
        dist=np.linalg.norm(a-n)
        #print("negative distance is ",dist)
        if(dist>32):
            forg_passed=forg_passed
          #  print("0")
        else:
            forg_passed=forg_passed+1
            print("forg passed - ",dist, "file name is - ", neg[j])
          #  print("1")

print("base_forg_passed is ",forg_passed)
print("base_gen_flagged is ",gen_flagged)

When k is  93
gen flagged -  39.737213 file name is -  g_93_8.tif
gen flagged -  34.501015 file name is -  g_93_9.tif
gen flagged -  38.463253 file name is -  g_93_13.tif
gen flagged -  38.36292 file name is -  g_93_2.tif
gen flagged -  32.1999 file name is -  g_93_7.tif
forg passed -  29.536472 file name is -  fSk_93_5.tif
forg passed -  30.570347 file name is -  fSk_93_6.tif
forg passed -  31.852669 file name is -  fOh_93_3.tif
When k is  94
gen flagged -  36.251564 file name is -  g_94_24.tif
gen flagged -  32.285385 file name is -  g_94_1.tif
gen flagged -  33.990437 file name is -  g_94_16.tif
gen flagged -  34.064407 file name is -  g_94_8.tif
gen flagged -  32.225384 file name is -  g_94_22.tif
forg passed -  30.105415 file name is -  fSk_94_5.tif
forg passed -  26.909828 file name is -  fSk_94_2.tif
When k is  95
gen flagged -  32.40643 file name is -  g_95_1.tif
gen flagged -  32.409775 file name is -  g_95_21.tif
gen flagged -  35.18725 file name is -  g_95_12.tif
forg passed

In [0]:
forg_passed=0
gen_flagged=0
x = range(93,116)

for k in x: #the id of signatures you want to check
    print("When k is ", k)
    anc1,anc2,neg,pos=getfilest(k,gent,forgt)
    
    anchor1=returnimages(gent,anc1)
    anchor2=returnimages(gent,anc2)
    for i in range(len(pos)): #pos
        positive=returnimages(gent,pos[i])
        x=model.predict([anchor1,positive,anchor2])
        a1, p, a2 = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-p)
        
        #Taking two distances (two anchor images), and taking average between the two
        dist1=np.linalg.norm(a1-p)
        dist2=np.linalg.norm(a2-p)
        dist=(dist1+dist2)/2
        #print("positive distance is ",dist)
        if(dist>32):
        #  print("0")
            gen_flagged=gen_flagged+1
            print("gen flagged - ",dist, "file name is - ", pos[i])
            
        else:
            gen_flagged=gen_flagged
        #   print("1")
        
    for j in range(len(neg)): #neg
        negative=returnimages(forgt,neg[j])
        x=model.predict([anchor1,negative,anchor2])
        a1, n, a2 = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-n)
        dist1=np.linalg.norm(a1-n)
        dist2=np.linalg.norm(a2-n)
        #print("negative distance is ",dist)
        dist=(dist1+dist2)/2
        if(dist>32):
            forg_passed=forg_passed
          #  print("0")
        else:
            forg_passed=forg_passed+1
            print("forg passed - ",dist, "file name is - ", neg[j])
          #  print("1")

print("forg_passed is ",forg_passed)
print("gen_flagged is ",gen_flagged)



### This is the code for the base model - duplicated with variables changed ###
forg_passed=0
gen_flagged=0
x = range(93,116)

for k in x: #the id of signatures you want to check
    print("When k is ", k)
    anc1,anc2,neg,pos=getfilest(k,gent,forgt)
    
    anchor1=returnimages(gent,anc1)
    anchor2=returnimages(gent,anc2)
    for i in range(len(pos)): #pos
        positive=returnimages(gent,pos[i])
        x=baseModel.predict([anchor1,positive,anchor2])
        a1, p, a2 = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-p)
        
        #Taking two distances (two anchor images), and taking average between the two
        dist1=np.linalg.norm(a1-p)
        dist2=np.linalg.norm(a2-p)
        dist=(dist1+dist2)/2
        #print("positive distance is ",dist)
        if(dist>32):
        #  print("0")
            gen_flagged=gen_flagged+1
            print("gen flagged - ",dist, "file name is - ", pos[i])
            
        else:
            gen_flagged=gen_flagged
        #   print("1")
        
    for j in range(len(neg)): #neg
        negative=returnimages(forgt,neg[j])
        x=baseModel.predict([anchor1,negative,anchor2])
        a1, n, a2 = x[0,0:512], x[0,512:1024], x[0,1024:1536]
        #dist=sum(a-n)
        dist1=np.linalg.norm(a1-n)
        dist2=np.linalg.norm(a2-n)
        #print("negative distance is ",dist)
        dist=(dist1+dist2)/2
        if(dist>32):
            forg_passed=forg_passed
          #  print("0")
        else:
            forg_passed=forg_passed+1
            print("forg passed - ",dist, "file name is - ", neg[j])
          #  print("1")

print("base_forg_passed is ",forg_passed)
print("base_gen_flagged is ",gen_flagged)

When k is  93
gen flagged -  32.83857345581055 file name is -  g_93_11.tif
gen flagged -  38.9965934753418 file name is -  g_93_8.tif
gen flagged -  34.26652908325195 file name is -  g_93_9.tif
gen flagged -  37.781044006347656 file name is -  g_93_13.tif
gen flagged -  38.423187255859375 file name is -  g_93_2.tif
forg passed -  29.83177375793457 file name is -  fSk_93_5.tif
forg passed -  29.039974212646484 file name is -  fSk_93_6.tif
When k is  94
gen flagged -  35.56007766723633 file name is -  g_94_24.tif
gen flagged -  32.189491271972656 file name is -  g_94_16.tif
gen flagged -  33.1716194152832 file name is -  g_94_22.tif
forg passed -  31.166643142700195 file name is -  fSk_94_5.tif
forg passed -  28.653966903686523 file name is -  fSk_94_2.tif
When k is  95
gen flagged -  34.44581604003906 file name is -  g_95_12.tif
forg passed -  28.562679290771484 file name is -  fSk_95_4.tif
forg passed -  27.947525024414062 file name is -  fSk_95_1.tif
forg passed -  29.097179412841797 